In [ ]:
import logging
import yaml

import numpy as np
from matplotlib import pyplot as plt
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import asyncio

# import palpy

from lsst.ts import salobj

from lsst.ts.standardscripts.auxtel.attcs import ATTCS

from lsst.ts.idl.enums import ATPtg

%matplotlib inline

# Start up

In [ ]:
from astropy.utils import iers
iers.conf.auto_download = False

In [ ]:
d = salobj.Domain()

In [ ]:
atmcs = salobj.Remote(d, "ATMCS")
atptg = salobj.Remote(d, "ATPtg")
ataos = salobj.Remote(d, "ATAOS")
atpne = salobj.Remote(d, "ATPneumatics")
athex = salobj.Remote(d, "ATHexapod")
atdome = salobj.Remote(d, "ATDome", index=1)
atdomtraj = salobj.Remote(d, "ATDomeTrajectory")

In [ ]:
await asyncio.gather(atmcs.start_task, 
                     atptg.start_task,
                     ataos.start_task,
                     atpne.start_task,
                     athex.start_task,
                     atdome.start_task,
                     atdomtraj.start_task)

## ATMCS

In [ ]:
await asyncio.sleep(5)

In [ ]:
await salobj.set_summary_state(atmcs, salobj.State.ENABLED, timeout=120)

## ATPtg

In [ ]:
await atptg.tel_timeAndDate.next(flush=True, timeout=5)
await asyncio.sleep(2.)
await salobj.set_summary_state(atptg, salobj.State.ENABLED)

In [ ]:
for i in range(10):
    time = await atptg.tel_timeAndDate.next(flush=True, timeout=5)
    print(time)

In [ ]:
await atptg.cmd_pointLoadModel.set_start(pointingFile='/home/saluser/repos/ts_pointing_common/install/data/at.mod')

In [ ]:
await atptg.cmd_pointLoadModel.set_start(pointingFile='auxtel_20190911.dat')
await atptg.cmd_pointLoadModel.set_start(pointingFile='/home/saluser/auxtel_20190925_01.mod')

In [ ]:
ptm = await atptg.evt_pointingModel.next(flush=False, timeout=5)

In [ ]:
ptm.pointingModelTermNames, ptm.pointingModelTermValues

In [ ]:
await salobj.set_summary_state(atptg, salobj.State.STANDBY)

## ATAOS

In [ ]:
await ataos.evt_heartbeat.next(flush=True)
await ataos.cmd_setLogLevel.set_start(level=logging.DEBUG, timeout=10)
# await salobj.set_summary_state(ataos, salobj.State.STANDBY)
await salobj.set_summary_state(ataos, salobj.State.ENABLED, settingsToApply="measured_20190908_redacted.yaml")

In [ ]:
await ataos.cmd_enableCorrection.set_start(m1=True, timeout=10)

In [ ]:
await ataos.cmd_disableCorrection.set_start(m1=True, timeout=10)

In [ ]:
await salobj.set_summary_state(ataos, salobj.State.STANDBY)

## ATPneumatics

In [ ]:
await asyncio.sleep(1.)
await salobj.set_summary_state(atpne, salobj.State.ENABLED)

In [ ]:
await salobj.set_summary_state(atpne, salobj.State.STANDBY)

In [ ]:
await atpne.cmd_closeMasterAirSupply.start(timeout=30)

In [ ]:
await atpne.cmd_closeInstrumentAirValve.start(timeout=30)

In [ ]:
await atpne.cmd_openM1Cover.start(timeout=30)

## ATDome

In [ ]:
await asyncio.sleep(5)
await salobj.set_summary_state(atdome, salobj.State.ENABLED, settingsToApply="test.yaml")

In [ ]:
await atdome.cmd_moveAzimuth.set_start(azimuth=200., timeout=30)

In [ ]:
await atdome.cmd_homeAzimuth.start()

In [ ]:
await atdome.cmd_openShutter.start()

In [ ]:
await atdome.cmd_closeShutter.start()

In [ ]:
await atdome.cmd_stopMotion.start()

In [ ]:
err = await atdome.evt_errorCode.next(flush=False, timeout=1)
err = atdome.evt_errorCode.get()

In [ ]:
err.errorReport

In [ ]:
await atdome.cmd_moveShutterMainDoor.set_start(open=True, timeout=10)

In [ ]:
await atdome.cmd_moveShutterMainDoor.set_start(open=False, timeout=10)

In [ ]:
await asyncio.sleep(5)
await salobj.set_summary_state(atdome, salobj.State.STANDBY)

## ATDomeTrajectory

In [ ]:
await salobj.set_summary_state(atdomtraj, salobj.State.ENABLED)

## ATHexapod

In [ ]:
await salobj.set_summary_state(athex, salobj.State.ENABLED, settingsToApply="Default1")

In [ ]:
tel = await athex.evt_positionUpdate.next(flush=False, timeout=2)

In [ ]:
tel.positionX, tel.positionY, tel.positionZ, tel.positionU, tel.positionV

In [ ]:
await athex.cmd_moveToPosition.set_start(x=-4., y=1.,z=0.35, u=0.35, v=0.22)

In [ ]:
await atptg.cmd_offsetAzEl.set_start(el=-15., az=0., num=1)

In [ ]:
await atptg.cmd_offsetAzEl.set_start(el=15., az=0., num=1)

In [ ]:
await atptg.cmd_offsetAzEl.set_start(el=0., az=-15., num=1)

In [ ]:
await salobj.set_summary_state(athex, salobj.State.STANDBY)

## Slewing/Tracking

In [ ]:
attcs = ATTCS(atmcs=atmcs, 
              atptg=atptg, 
              ataos=ataos, 
              atpneumatics=atpne, 
              athexapod=athex, 
              atdome=atdome, 
              atdometrajectory=atdomtraj, 
              check={"atpneumatics": False, "athexapod": False, "atdome": True, "atdometrajectory": True})

In [ ]:
location = EarthLocation.from_geodetic(lon=-70.747698*u.deg,
                                       lat=-30.244728*u.deg,
                                       height=2663.0*u.m)

In [ ]:
ra = Angle("21:15:49.4319197", unit=u.hour)
dec = Angle("+05:14:52.243012", unit=u.deg)
target_name="Alf Equ"
radec = ICRS(ra, dec)

In [ ]:

radec = ICRS(ra, dec)

# Figure out what is the rotPA that sets nasmith rotator close to zero.
time_data = await atptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
print(curr_time_atptg)
coord_frame_altaz = AltAz(location=location, obstime=curr_time_atptg)
alt_az = radec.transform_to(coord_frame_altaz)

print("slew...")
# await atmcs.cmd_startTracking.start(timeout=10)
await attcs.slew(ra=radec.ra.hour, 
                 dec=radec.dec.deg,
                 rotPA=180.-alt_az.alt.deg,
                 rot_frame=ATPtg.RotFrame.FIXED,
                 rot_mode=ATPtg.RotMode.FIELD)

In [ ]:
await atptg.cmd_offsetAzEl.set_start(el=30., az=0.,num=1)

In [ ]:
await atptg.cmd_offsetAzEl.set_start(el=-250.)

In [ ]:
await atptg.cmd_offsetAzEl.set_start(el=0., az=-30., num=1)

In [ ]:
await atptg.cmd_offsetAbsorb.set_start(num=2,timeout=10)

In [ ]:
20*360/24.

In [ ]:
await atptg.cmd_exitControl.start()

In [ ]:
await atmcs.cmd_startTracking.start(timeout=30)

In [ ]:
# Figure out what is the rotPA that sets nasmith rotator close to zero.
time_data = await atptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
print(curr_time_atptg)
coord_frame_altaz = AltAz(location=location, obstime=curr_time_atptg)
alt_az = radec.transform_to(coord_frame_altaz)

print("slew...")
# await atmcs.cmd_startTracking.start(timeout=10)
await atptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    targetInstance=ATPtg.TargetInstances.CURRENT,
    frame=ATPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=radec.ra.hour,
    declination=radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=180.-alt_az.alt.deg,
    rotFrame=ATPtg.RotFrame.FIXED,
    rotMode=ATPtg.RotMode.FIELD,
    timeout=10
)

In [ ]:
off = await atptg.tel_guidingAndOffsets.next(flush=True, timeout=5)

In [ ]:
off.userOffsetDec, off.handsetOffsetDec, off.userOffsetRA, off.handsetOffsetRA

# alf Sco

In [ ]:
ra = Angle("16:29:24.45970"  , unit=u.hour)
dec = Angle("-26:25:55.2094", unit=u.deg)
target_name="alf Sco"
radec = ICRS(ra, dec)



In [ ]:
# Figure out what is the rotPA that sets nasmith rotator close to zero.
time_data = await atptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
print(curr_time_atptg)
coord_frame_altaz = AltAz(location=location, obstime=curr_time_atptg)
alt_az = radec.transform_to(coord_frame_altaz)

print("slew...")
# await atmcs.cmd_startTracking.start(timeout=10)
await atptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    targetInstance=ATPtg.TargetInstances.CURRENT,
    frame=ATPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=radec.ra.hour,
    declination=radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=180.-alt_az.alt.deg,
    rotFrame=ATPtg.RotFrame.FIXED,
    rotMode=ATPtg.RotMode.FIELD,
    timeout=10
)

## kap Sco

In [ ]:
ra=
dec=

In [ ]:
ra = Angle("17:42:29.27520" , unit=u.hour)
dec = Angle("-39:01:47.9391", unit=u.deg)
target_name="Kap Sco"
radec = ICRS(ra, dec)

In [ ]:
# Figure out what is the rotPA that sets nasmith rotator close to zero.
time_data = await atptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
print(curr_time_atptg)
coord_frame_altaz = AltAz(location=location, obstime=curr_time_atptg)
alt_az = radec.transform_to(coord_frame_altaz)

print("slew...")
# await atmcs.cmd_startTracking.start(timeout=10)
await atptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    targetInstance=ATPtg.TargetInstances.CURRENT,
    frame=ATPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=radec.ra.hour,
    declination=radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=180.-alt_az.alt.deg,
    rotFrame=ATPtg.RotFrame.FIXED,
    rotMode=ATPtg.RotMode.FIELD,
    timeout=10
)

In [ ]:
await atptg.cmd_stopTracking.start(timeout=10)

In [ ]:
# off.userOffsetDec, off.handsetOffsetDec, off.userOffsetRA, off.handsetOffsetRA
await atptg.cmd_offsetRADec.set_start(type=1, 
                                      off1=off.userOffsetRA+off.handsetOffsetRA,
                                      off2=off.userOffsetDec+off.handsetOffsetDec,
                                     num=0)

In [ ]:
await atptg.cmd_offsetAzEl.set_start(el=0., az=-250., num=1)

In [ ]:
await atptg.cmd_offsetAzEl.set_start(el=0., az=10., num=1)

In [ ]:
await atptg.cmd_offsetAzEl.set_start(el=-15., az=0., num=1)

# GRID

In [ ]:
async def wait_center():
    while True:
        opt = input("Center telescope and choose action (press ?<enter>, for list of actions): ")
        
        if opt == 'n':
            print("Next point in the grid...")
            return 1
        elif opt == 'p':
            print("Previous point in the grid...")
            return -1
        elif opt == 't':
            print("Stopping test...")
            return 0
        elif opt == '?':
            print("""Options are:
            n - Go to next point in the grid.
            p - Go to previous point in the grid.
            t - Terminate test.
            """)
        else:
            print("Next point in the grid...")
            return 1

In [ ]:
dalt_grid = np.arange(-1200., 1200, 240.)-2600.
daz_grid = np.arange(-1200, 1200, 240)-1200.

In [ ]:
await atptg.cmd_offsetAzEl.set_start(el=-3000., 
                                     az=-1200., 
                                     num=0)

In [ ]:
grid_alt = np.zeros(len(dalt_grid)*len(daz_grid))
grid_az = np.zeros(len(dalt_grid)*len(daz_grid))

for i in range(len(daz_grid)):
    grid_alt[i*len(dalt_grid):(i+1)*len(dalt_grid)] += dalt_grid[::(-1)**i]
    grid_az[i*len(dalt_grid):(i+1)*len(dalt_grid)] += daz_grid[i]

In [ ]:
plt.plot(grid_alt, grid_az, ':')
plt.plot(grid_alt, grid_az, '|')
plt.plot(grid_alt[0], grid_az[0], 'bo')
plt.plot(grid_alt[-1], grid_az[-1], 'go')

In [ ]:
start_from=0

In [ ]:
print(f"Grid has {len(grid_alt)} pointings...")
print(f"Starting from {start_from}")

i = start_from
stop_at = 0

while i < len(grid_alt):
    
    print(f"GRID[{i}]: {grid_alt[i]} x {grid_az[i]}")

    await atptg.cmd_offsetAzEl.set_start(el=grid_alt[i], 
                                         az=grid_az[i], 
                                         num=0)
    
    ret_val = await wait_center()

    if ret_val == 0:
        break
        
    i += ret_val
    
    if i < 0:
        print(f"Unwrapping index! (i={i})")
        i = 0

In [ ]:
i=36
await atptg.cmd_offsetAzEl.set_start(el=grid_alt[i]*60.*60., 
                                     az=grid_az[i]*60*60, 
                                     num=0)



# Parking

In [ ]:
await atpne.cmd_closeM1Cover.start(timeout=30)

In [ ]:
await salobj.set_summary_state(atdomtraj, salobj.State.DISABLED)

In [ ]:
await asyncio.sleep(2.5)
await salobj.set_summary_state(atdome, salobj.State.ENABLED)
# await salobj.set_summary_state(atptg, salobj.State.ENABLED)

In [ ]:
await asyncio.sleep(2.5)
# atptg.evt_inPosition.flush()
await atptg.cmd_azElTarget.set_start(azDegs=90., elDegs=80.)
# while True:
#     inp = await atptg.evt_inPosition.next(flush=False, timeout=120.)
#     if inp.inPosition:
#         break

In [ ]:
err = await atptg.evt_errorCode.next(flush=False, timeout=1)
err = atptg.evt_errorCode.get()

In [ ]:
err.errorReport

In [ ]:
await atmcs.cmd_startTracking.start()

In [ ]:
await atptg.cmd_stopTracking.start()

In [ ]:
await atdome.cmd_closeShutter.start(timeout=30)

In [ ]:
await atptg.cmd_stopTracking.start()

In [ ]:
await atdome.cmd_moveAzimuth.set_start(azimuth=200.)

In [ ]:
await asyncio.gather(salobj.set_summary_state(atmcs, salobj.State.STANDBY),
                     salobj.set_summary_state(atptg, salobj.State.STANDBY),
                     salobj.set_summary_state(atdome, salobj.State.STANDBY),
                     salobj.set_summary_state(athex, salobj.State.STANDBY),
                     salobj.set_summary_state(atpne, salobj.State.STANDBY),
                     salobj.set_summary_state(ataos, salobj.State.STANDBY))

In [ ]:
await asyncio.sleep(5)
await salobj.set_summary_state(atdome, salobj.State.DISABLED)

In [ ]:
print(salobj.State(ss.summaryState))

In [ ]:
await asyncio.sleep(10.)

In [ ]:
await salobj.set_summary_state(atmcs, salobj.State.STANDBY, timeout=120)

In [ ]:
def mainValveCallback(data):
    print(data.state)

In [ ]:
atpne.evt_mainValveState.callback = mainValveCallback

In [ ]:
atpne.evt_mainValveState.callback = None

In [ ]:
await asyncio.sleep(10.)
# mv = atpne.evt_mainValveState.get()

In [ ]:
mv.state

In [ ]:
await atpne.cmd_closeMasterAirSupply.start(timeout=5)

In [ ]:
mv = await atpne.evt_mainValveState.next(flush=False, timeout=2)

In [ ]:
print(mv)

In [ ]:
mv.state, mv.private_sndStamp

In [ ]:
mv = atpne.evt_mainValveState.get()

In [ ]:
print(mv)

In [ ]:
mv.state, mv.private_sndStamp

In [ ]:
await atpne.cmd_openMasterAirSupply.start(timeout=5)

In [ ]:
mv = await atpne.evt_instrumentState.next(flush=False, timeout=2)

In [ ]:
mv.state

In [ ]:
await atpne.cmd_openInstrumentAirValve.start(timeout=5)

In [ ]:
try:
    await atpne.cmd_m1OpenAirValve.start(timeout=10)
except salobj.AckError as e:
    if e.ackcmd.ack == salobj.SalRetCode.CMD_NOPERM:
        print(e)
    else:
        raise e

In [ ]:
await asyncio.sleep(10.)

In [ ]:
await atdomtraj.cmd_start.start()

In [ ]:
await asyncio.sleep(1.)
atpne.evt_m1State.flush()
await atpne.cmd_m1OpenAirValve.start(timeout=10)
m1state = await atpne.evt_m1State.next(flush=False, timeout=10)
print(m1state.state)

In [ ]:
await asyncio.sleep(1.)
atpne.evt_m1State.flush()
await atpne.cmd_m1CloseAirValve.start(timeout=10)
m1state = await atpne.evt_m1State.next(flush=False, timeout=10)
print(m1state.state)

In [ ]:
await asyncio.sleep(5.)
atpne.evt_instrumentState.flush()
await atpne.cmd_openInstrumentAirValve.start(timeout=10)
m1state = await atpne.evt_instrumentState.next(flush=False, timeout=10)
print(m1state.state)
await asyncio.sleep(5.)

In [ ]:
await asyncio.sleep(5.)
atpne.evt_instrumentState.flush()
await atpne.cmd_closeInstrumentAirValve.start(timeout=10)
m1state = await atpne.evt_instrumentState.next(flush=False, timeout=10)
print(m1state.state)
await asyncio.sleep(5.)

In [ ]:
await atpne.cmd_openMasterAirSupply.start()

In [ ]:
await asyncio.sleep(1.)
# hexp = await athex.evt_positionUpdate.next(flush=False, timeout=1)
hexp = athex.evt_positionUpdate.get()

In [ ]:
hexp.positionX,hexp.positionY,hexp.positionZ,hexp.positionU,hexp.positionV

In [ ]:
await athex.cmd_moveToPosition.set_start(x=-4., y=1.,z=0.35, u=0.35, v=0.22)

In [ ]:
# await asyncio.sleep(5.)
pt1 = await atptg.evt_pointingModel.next(flush=True, timeout=30)
pt1 = atptg.evt_pointingModel.get()

In [ ]:
pt.pointingModelTermNames

In [ ]:
pt.pointingModelTermValues

In [ ]:
pt1.pointingModelTermNames

In [ ]:
pt1.pointingModelTermValues

In [ ]:
off = await atptg.tel_guidingAndOffsets.next(flush=False, timeout)

In [ ]:
await atptg.cmd_azElTarget.set_start(azDegs=0., elDegs=80.)

In [ ]:
await atptg.cmd_stopTracking.start()